In [1]:
pwd

'/Users/jeremy/Desktop/nonsense/cbb_modeling'

In [2]:
import pandas as pd
import numpy as np

### Load Data
Data is expected to be in a /data folder in same directory as this notebook

In [3]:
teams = pd.read_csv("/Users/jeremy/Desktop/nonsense/cbb_modeling/data/MTeams.csv")
teams.shape

(380, 4)

In [4]:
seasons = pd.read_csv("/Users/jeremy/Desktop/nonsense/cbb_modeling/data/MSeasons.csv")
seasons.shape

(41, 6)

In [37]:
tourney_compact_results = pd.read_csv('/Users/jeremy/Desktop/nonsense/cbb_modeling/data/MNCAATourneyCompactResults.csv')
tourney_compact_results.shape

(2518, 8)

In [6]:
tourney_detailed_results = pd.read_csv('/Users/jeremy/Desktop/nonsense/cbb_modeling/data/MNCAATourneyDetailedResults.csv')
tourney_detailed_results.shape

(1382, 34)

In [7]:
reg_season_detailed_results = pd.read_csv('/Users/jeremy/Desktop/nonsense/cbb_modeling/data/MRegularSeasonDetailedResults.csv')
reg_season_detailed_results.shape

(116723, 34)

In [8]:
reg_season_compact_results = pd.read_csv('/Users/jeremy/Desktop/nonsense/cbb_modeling/data/MRegularSeasonCompactResults.csv')
reg_season_compact_results.shape

(190771, 8)

In [38]:
### Add WTeam names
tourney_compact_results = pd.merge(left=tourney_compact_results, 
                                   right=teams[['TeamID', 'TeamName']], 
                                   left_on='WTeamID',
                                   right_on='TeamID',
                                   how='left')

tourney_compact_results = tourney_compact_results.rename(columns={"TeamName":'WTeamName'})

In [39]:
tourney_compact_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,TeamID,WTeamName
0,1985,136,1116,63,1234,54,N,0,1116,Arkansas
1,1985,136,1120,59,1345,58,N,0,1120,Auburn
2,1985,136,1207,68,1250,43,N,0,1207,Georgetown
3,1985,136,1229,58,1425,55,N,0,1229,Illinois St
4,1985,136,1242,49,1325,38,N,0,1242,Kansas


In [40]:
tourney_compact_results[tourney_compact_results['Season']==2024].sort_values('DayNum', ascending=False).head(7)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,TeamID,WTeamName
2517,2024,154,1163,75,1345,60,N,0,1163,Connecticut
2516,2024,152,1345,63,1301,50,N,0,1345,Purdue
2515,2024,152,1163,86,1104,72,N,0,1163,Connecticut
2514,2024,146,1345,72,1397,66,N,0,1345,Purdue
2513,2024,146,1301,76,1181,64,N,0,1301,NC State
2512,2024,145,1163,77,1228,52,N,0,1163,Connecticut
2511,2024,145,1104,89,1155,82,N,0,1104,Alabama


In [13]:
reg_season_compact_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [16]:
reg_season_compact_results['Season'].max()

2025

### Create calculated fields for features

In [ ]:
df['WPoss'] = (df['WFGA'] - df['WOR'] + df['WTO'] + (0.475*df['WFTA']) )
df['LPoss'] = (df['LFGA'] - df['LOR'] + df['LTO'] + (0.475*df['LFTA']) )
df['TotPoss'] = df['WPoss'] + df['LPoss']
df['WPPP'] = (df['WScore'] / ((df['WPoss'] + df['LPoss'])/2) )
df['LPPP'] = (df['LScore'] / ((df['WPoss'] + df['LPoss'])/2) )

## Create training data set

In [ ]:
train_df = tourney_detailed_results.copy()